## <center>РОЗВ’ЯЗАННЯ СИСТЕМ ЛІНІЙНИХ АЛГЕБРАЇЧНИХ РІВНЯНЬ. МЕТОД ГАУСА</center> 
 **Мета роботи:** вивчення алгоритмів для розв’язання систем лінійних алгебраїчних рівнянь (СЛАР) методом Гауса.
 
 
**Що зробити:** скласти процедуру для розв’язання СЛАР методом Гауса, яка б у випадку невиродженої системи знаходила її розв’язок, а для виродженої системи видавала відповідне попередження. Впевнитись в коректності роботи процедури, підставляючи в СЛАР отримані розв’язки і обраховуючи нев’язки. Додатково - передбачити оцінку числа обумовленості матриці системи.

![](tasks.png)

**Метод Гаусса** – это метод последовательного исключения неизвестных.Суть метода Гаусса состоит в преобразовании СЛАУ к системе с треугольной матрицей, из которой затем последовательно (обратным ходом) получаются значения всех неизвестных. 

In [1]:
import numpy as np
import math

def Gauss(Ab): # На вход подается расширенная матрица СЛАУ(коэффиенты при неизвестных и вектор-столбец свободных членов)
    N = len(Ab) # N - количество уравнений(предполагаем что количество уравнений равно количеству неизвестных)
    
    print("Solving system: ")
    print(Ab)
    
    for i in range(0,N-1): # Прямой ход метода Гаусса, приведение матрицы СЛАУ к верхнетреугольному виду.
        
        for j in range(i+1,N): # во всех строках начиная c i+1-ой, обнуляем коэффициент при i-ом неизвестном, путем вычитания
            Ab[j] += Ab[i]*(- Ab[j,i] / Ab[i,i] ) #i-ой строки, из всех нижележащих
            
        print("--------------------------------")
        print("{0} step".format(i+1))
        print(Ab)
        
    x = np.zeros(N) # обратный ход метода Гаусса, нахождение 
    x[N-1] = Ab[N-1,N]/Ab[N-1,N-1]
    for i in range(N-2,-1, -1): #находим неизвестные последовательно снизу вверх
        s = 0
        for j in range(i+1, N):
            s = s + x[j]*Ab[i, j]
        x[i] = (Ab[i, N] - s) / Ab[i,i]
    return x

Решим первую систему уравнений

In [2]:
Ab1 = np.array([\
    [ 2,  0,  3,   1,  20],\
    [-4,  3, -4,  -2, -34],\
    [ 4,  7,  9,   1,  48],\
    [ 5,  7,  0,   8,  97],\
])
Ab1 = Ab1.astype(float)
np.set_printoptions(precision = 3,suppress=True) #вывод с точностью до 3-х знаков после запятой

x1 = Gauss(Ab1.copy())
print("\nX = ", x1)

Solving system: 
[[  2.   0.   3.   1.  20.]
 [ -4.   3.  -4.  -2. -34.]
 [  4.   7.   9.   1.  48.]
 [  5.   7.   0.   8.  97.]]
--------------------------------
1 step
[[  2.    0.    3.    1.   20. ]
 [  0.    3.    2.    0.    6. ]
 [  0.    7.    3.   -1.    8. ]
 [  0.    7.   -7.5   5.5  47. ]]
--------------------------------
2 step
[[  2.      0.      3.      1.     20.   ]
 [  0.      3.      2.      0.      6.   ]
 [  0.      0.     -1.667  -1.     -6.   ]
 [  0.      0.    -12.167   5.5    33.   ]]
--------------------------------
3 step
[[  2.      0.      3.      1.     20.   ]
 [  0.      3.      2.      0.      6.   ]
 [  0.      0.     -1.667  -1.     -6.   ]
 [  0.      0.      0.     12.8    76.8  ]]

X =  [ 7.  2.  0.  6.]


Посчитаем невязку решения:

In [3]:
def residual(Ab,x):
    return np.matmul(Ab[:,:-1],x) - Ab[:,-1]

In [4]:
r = residual(Ab1,x1)
print(r)

[ 0.  0.  0.  0.]


   В процедуре на каждом шаге предполагалось, что ведущий элемент отличен от нуля. Если это не так, то в качестве ведущего можно использовать любой другой элемент, переставив соответствующие строки матрицы. Для избежания ошибок округления при делении на очень маленькие по абсолютной величине элементы, используется модификация метода Гаусса с выбором главного элемента, когда на каждом шаге выбирается строка с максимальным по значению ведущим элементом. 

In [5]:
def Gauss_modified(Ab, printSteps=False): #на вход подается расширенная матрица, вместо столбца свободных членов можно передать
    if printSteps:
        print("Solving system: ")              #любую матрицу, например единичную для поиска обратной.
        print(Ab)
    
    det = 0 #добавим вычисление определителя
    permutations = 0 #количество перестановок строк
    
    N = len(Ab)
    x = np.zeros(( N, Ab.shape[1] - N))
    for i in range(0,N-1): #Прямой ход
        max_element_index = np.argmax(np.abs(Ab[i:,i])) #ищем строку с максимальным ведущим элементом
        if math.isclose(Ab[max_element_index+i,i],0):
            raise np.linalg.LinAlgError('Singular matrix') #все ведущие элементы нулевые -> матрица вырожденная
        
        if (printSteps):
            print("--------------------------------")
            print("{0} step".format(i+1))
            print("Switch {0} and {1} rows".format( i+1, max_element_index+i+1))
        
        Ab[i,:], Ab[max_element_index+i,:] = Ab[max_element_index+i, :], Ab[i,:].copy() #перестановка строк
        permutations+=1
        if (printSteps):
            print(Ab)
            print("Eliminate {0} variable".format(i+1)) #Исключаем i-ое неизвестное
            
        for j in range(i+1,N):
            Ab[j] += Ab[i]*(- Ab[j,i] / Ab[i,i] )
        if (printSteps):
            print(Ab)
        
    x[N-1] = Ab[N-1,N:]/Ab[N-1,N-1]

    for i in range(N-2,-1, -1):
        s = np.zeros( (1, x.shape[1]))

        for j in range(i+1, N):
            s = s + x[j]*Ab[i, j]
        x[i] = (Ab[i, N:] - s) / Ab[i,i]
        
    det = np.prod(Ab.diagonal())*(-1)**permutations
    
    return x,  det

Решим вторую систему уравнений:

In [6]:
Ab2 = np.array([\
    [-7,  -6,   -6,   6,  144 ],\
    [ 7,   6,    8,  -13, -170],\
    [ 4,  17,  -16,   10,  21 ],\
    [-5,  18,   19,   0,  -445],\
])
Ab2 = Ab2.astype(float)
x2, det = Gauss_modified(Ab2.copy(),printSteps = True)
print("\nX = ", x2.reshape(4))

Solving system: 
[[  -7.   -6.   -6.    6.  144.]
 [   7.    6.    8.  -13. -170.]
 [   4.   17.  -16.   10.   21.]
 [  -5.   18.   19.    0. -445.]]
--------------------------------
1 step
Switch 1 and 1 rows
[[  -7.   -6.   -6.    6.  144.]
 [   7.    6.    8.  -13. -170.]
 [   4.   17.  -16.   10.   21.]
 [  -5.   18.   19.    0. -445.]]
Eliminate 1 variable
[[  -7.      -6.      -6.       6.     144.   ]
 [   0.       0.       2.      -7.     -26.   ]
 [   0.      13.571  -19.429   13.429  103.286]
 [   0.      22.286   23.286   -4.286 -547.857]]
--------------------------------
2 step
Switch 2 and 4 rows
[[  -7.      -6.      -6.       6.     144.   ]
 [   0.      22.286   23.286   -4.286 -547.857]
 [   0.      13.571  -19.429   13.429  103.286]
 [   0.       0.       2.      -7.     -26.   ]]
Eliminate 2 variable
[[  -7.      -6.      -6.       6.     144.   ]
 [   0.      22.286   23.286   -4.286 -547.857]
 [   0.      -0.     -33.609   16.038  436.917]
 [   0.       0.       2.

Невязка:

In [7]:
r = residual(Ab2,x2.reshape(4))
print(r)


[ 0.  0.  0. -0.]


Решим 3 систему:

In [8]:
Ab3 = np.array([\
    [ 3,  -2,    -7,   -1,    2],\
    [ 7, -10,    -5,    1,   28],\
    [ 4,   0,   -15,   -9,  -21],\
    [-8,   8,    13,    4,  -11],\
])
Ab3 = Ab3.astype(float)
try:
    x3, det = Gauss_modified(Ab3.copy(),printSteps = True)
    print("\nX = ", x.reshape(4))
except np.linalg.LinAlgError:
    print ("MATRIX IS SINGULAR")


Solving system: 
[[  3.  -2.  -7.  -1.   2.]
 [  7. -10.  -5.   1.  28.]
 [  4.   0. -15.  -9. -21.]
 [ -8.   8.  13.   4. -11.]]
--------------------------------
1 step
Switch 1 and 4 rows
[[ -8.   8.  13.   4. -11.]
 [  7. -10.  -5.   1.  28.]
 [  4.   0. -15.  -9. -21.]
 [  3.  -2.  -7.  -1.   2.]]
Eliminate 1 variable
[[ -8.      8.     13.      4.    -11.   ]
 [  0.     -3.      6.375   4.5    18.375]
 [  0.      4.     -8.5    -7.    -26.5  ]
 [  0.      1.     -2.125   0.5    -2.125]]
--------------------------------
2 step
Switch 2 and 3 rows
[[ -8.      8.     13.      4.    -11.   ]
 [  0.      4.     -8.5    -7.    -26.5  ]
 [  0.     -3.      6.375   4.5    18.375]
 [  0.      1.     -2.125   0.5    -2.125]]
Eliminate 2 variable
[[ -8.     8.    13.     4.   -11.  ]
 [  0.     4.    -8.5   -7.   -26.5 ]
 [  0.     0.     0.    -0.75  -1.5 ]
 [  0.     0.     0.     2.25   4.5 ]]
MATRIX IS SINGULAR


**LU-разложение** — представление матрицы A в виде произведения двух матриц, **A=LU**, где **L** — нижняя треугольная матрица, а **U** — верхняя треугольная матрица.

**LU-разложение** используется для решения систем линейных уравнений, обращения матриц и вычисления определителя.

In [9]:
def LU_Factor(A):
    LU = np.zeros(A.shape)
    N = len(LU)
    np.fill_diagonal(LU,1)
    LU[0] = A[0]
    LU[1:,0] = A[1:,0]/LU[0,0]
    
    for i in range(1, N):
        for col in range(i,N): #элементы U
            LU[i, col] = A[i,col] - np.dot(LU[i,:i], LU[:i,col])
        for row in range(i+1,N): #элементы L
            LU[row,i] = (A[row,i] - np.dot(LU[row, :i], LU[:i,i]))/LU[i,i]
    return LU

Полученное **LU-разложение** матрицы **A** (матрица коэффициентов системы) может быть использовано для решения 
семейства систем линейных уравнений с различными векторами **b** в правой части:

   **Ax=b**
   
Если известно **LU-разложение** матрицы **A**,  **A=LU**, исходная система может быть записана как

 **LUx=b**
 
Эта система может быть решена в два шага. На первом шаге решается система

**Ly=b.**

Поскольку **L** — нижняя треугольная матрица, эта система решается непосредственно прямой подстановкой.

На втором шаге решается система

**Ux=y.**

Поскольку **U** — верхняя треугольная матрица, эта система решается непосредственно обратной подстановкой.

Используем **LU**-разложение для вычисления обратной матрицы. Для этого решаем систему уравнений **LUx=E**, где **E** - единичная матрицйа

In [10]:
def inverse_matrix_LU(M):
    LU = LU_Factor(M)
    L = np.tril(LU)
    np.fill_diagonal(L, 1)
    U = np.triu(LU)
    y, det = Gauss_modified(np.append(L,np.identity(len(L)),axis = 1)) # решаем систему Ly=E
    inverse, det = Gauss_modified(np.append(U, y, axis = 1))#решаем систему Ux=y 
    return inverse


In [11]:
A1_inv = inverse_matrix_LU(Ab1[:,:-1])
print(A1_inv)


[[-4.203 -1.641  0.672  0.031]
 [-0.703 -0.141  0.172  0.031]
 [ 2.055  0.711 -0.258 -0.047]
 [ 3.242  1.148 -0.57   0.078]]


Для невырожденной матрицы **LU-разложение** существует тогда и только тогда, когда все ее главные миноры не равны 0. Поскольку это условие, для матрицы коэффициентов второй системы уравнений не выполняется, для нахождение обратной матрицы, используем метод Гаусса, с расширенной матрицей **A|E**

In [12]:
def inverse_matrix_gauss(M):
    inverse, det = Gauss_modified(np.append(M, np.identity(len(M)),axis = 1))
    return inverse

In [13]:
inverse_matrix_gauss(Ab2[:,:-1])

array([[-0.241, -0.118, -0.008, -0.034],
       [ 0.053,  0.051,  0.034,  0.024],
       [-0.114, -0.079, -0.034,  0.021],
       [-0.175, -0.165, -0.01 ,  0.006]])

Числом обусловленности ***cond(A)*** оператора **A** называется число **cond(A) = ||A||\*||A^(-1)||**
Если число обусловленности оператора **A** мало́, то оператор называется хорошо обусловленным. Если же число обусловленности велико, то оператор называется плохо обусловленным. Чем меньше **cond(A)**, тем «лучше», то есть тем меньше погрешности решения будут относительно погрешностей в условии. Учитывая, что **cond(A)>=1**, то наилучшим числом обусловленности является 1.

In [14]:
def condition_number(M):
    M_inv = inverse_matrix_gauss(M)
    return np.linalg.norm(M,np.inf)*np.linalg.norm(M_inv, np.inf)

Число обусловленности 1-й системы:

In [15]:
print(Ab1[:,:-1])
print("Condition number: ", condition_number(Ab1[:,:-1]))

[[ 2.  0.  3.  1.]
 [-4.  3. -4. -2.]
 [ 4.  7.  9.  1.]
 [ 5.  7.  0.  8.]]
Condition number:  137.484375


Число обусловленности 2-й системы:

In [16]:
print(Ab2[:,:-1])
print("Condition number: ", condition_number(Ab2[:,:-1]))

[[ -7.  -6.  -6.   6.]
 [  7.   6.   8. -13.]
 [  4.  17. -16.  10.]
 [ -5.  18.  19.   0.]]
Condition number:  18.8151244597


Введем возмущение в правой части исходных СЛАУ:

In [17]:
Ab1[0,-1]*=1.01
print("1 system:\n", Ab1)
Ab2[0,-1]*=1.01
print("2 system:\n", Ab2)

1 system:
 [[  2.    0.    3.    1.   20.2]
 [ -4.    3.   -4.   -2.  -34. ]
 [  4.    7.    9.    1.   48. ]
 [  5.    7.    0.    8.   97. ]]
2 system:
 [[  -7.     -6.     -6.      6.    145.44]
 [   7.      6.      8.    -13.   -170.  ]
 [   4.     17.    -16.     10.     21.  ]
 [  -5.     18.     19.      0.   -445.  ]]


Решим СЛАУ с возмущением и посчитаем относительную погрешность решений:

In [18]:
x1_modified, det = Gauss_modified(Ab1.copy())
x2_modified, det = Gauss_modified(Ab2.copy())

In [19]:
print("X1 = ", x1, "X1_modified = ", x1_modified.reshape(4), "Error: ",\
          np.linalg.norm(x1-x1_modified.reshape(4),np.inf)/np.linalg.norm(x1,np.inf))
print("X2 = ", x2.reshape(4), "X2_modified = ", x2_modified.reshape(4), "Error: ",\
          np.linalg.norm(x2-x2_modified,np.inf)/np.linalg.norm(x2,np.inf))


X1 =  [ 7.  2.  0.  6.] X1_modified =  [ 6.159  1.859  0.411  6.648] Error:  0.120089285714
X2 =  [  0. -11. -13.  -0.] X2_modified =  [ -0.347 -10.924 -13.164  -0.252] Error:  0.0267094435047


Число обусловленности второй СЛАУ меньше, поэтому, как и следовало ожидать, погрешность решения значительно ниже - менее 3%